Import Dependencies

In [ ]:
import pandas as pd
import os
import datetime as dt
#import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
import bokeh
from bokeh.plotting import figure, output_file, show

#import spmfunctions.spmfunctions as spm

#%matplotlib inline

: 

In [7]:
from datetime import datetime

datetime.strptime("2025:08:02:23:00", "%Y:%m:%d:%H:%M")

datetime.datetime(2025, 8, 2, 23, 0)

In [2]:
os.chdir('../..')

In [3]:
os.getcwd()

'c:\\Users\\rhansen\\Documents\\Python\\SPMs'

In [ ]:

intersections=['Wainwright','Colchester','HobbleCreek','IslandWoods','Pine','Riverside','RiverValley','StLuke','Chinden', 'Fairview', 'Franklin', 'McMillan', 'Ustick','I84EB','I84WB']
yymms = ['2024_09', '2024_10', '2024_11', '2024_12', 
         '2025_01', '2025_02', '2025_03', '2025_04', 
         '2025_05', '2025_06', '2025_07', '2025_08']


results = []
for intx in intersections:
    print(f'Processing intersection {intx}...')
    pickles = [f for f in os.listdir(f'G:/Python/SPM_Data_Archive/ACHD_Data/ACHD_Eagle-{intx}/Data/Dataframes/') if f.endswith('.pkl')]

    selected_pickles = [f for f in pickles if any(yymm in f for yymm in yymms)]

    df_raw = pd.concat([pd.read_pickle(f'G:/Python/SPM_Data_Archive/ACHD_Data/ACHD_Eagle-{intx}/Data/Dataframes/' + f) for f in selected_pickles], ignore_index=True)
    df_raw = df_raw.drop_duplicates().reset_index(drop=True).sort_values(by='TS_start')
    df_pc = plan_changes(df_raw)
    df_pc['Intersection'] = intx

    results.append(df_pc)

df_res = pd.concat(results)
df_res.to_csv( './Intersections/ACHD_Eagle-All_2024-09_to_2025-08_Plan_Changes.csv')

Processing intersection Wainwright...
Processing intersection Colchester...
Processing intersection HobbleCreek...
Processing intersection IslandWoods...
Processing intersection Pine...
Processing intersection Riverside...
Processing intersection RiverValley...
Processing intersection StLuke...
Processing intersection Chinden...
Processing intersection Fairview...
Processing intersection Franklin...
Processing intersection McMillan...
Processing intersection Ustick...
Processing intersection I84EB...
Processing intersection I84WB...


OSError: Cannot save file into a non-existent directory: 'Intersections'

In [12]:

intersections=['Franklin & Aviation', 'Franklin & Smeed', 'Franklin & KCID', 'Franklin & Ward']
yymms = ['2025_09', '2025_10', '2025_11']

df_raws = dict()
results = []
for intx in intersections:
    print(f'Processing intersection {intx}...')
    pickles = [f for f in os.listdir(f'../../Intersections/{intx}/Data/Dataframes/') if f.endswith('.pkl')]

    selected_pickles = [f for f in pickles if any(yymm in f for yymm in yymms)]

    df_raw = pd.concat([pd.read_pickle(f'../../Intersections/{intx}/Data/Dataframes/' + f) for f in selected_pickles], ignore_index=True)
    df_raw = df_raw.drop_duplicates().reset_index(drop=True).sort_values(by='TS_start')
    df_raws[intx] = df_raw
    df_pc = plan_changes(df_raw)
    df_pc['Intersection'] = intx

    results.append(df_pc)

df_res = pd.concat(results)
df_res.to_csv( '../../Intersections/Franklin_Plan_Changes.csv')

Processing intersection Franklin & Aviation...
Processing intersection Franklin & Smeed...
Processing intersection Franklin & KCID...
Processing intersection Franklin & Ward...


PermissionError: [Errno 13] Permission denied: '../../Intersections/Franklin_Plan_Changes.csv'

In [4]:
os.getcwd()

'c:\\Users\\rhansen\\Documents\\Python\\SPMs\\Notebooks\\Snippets'

In [13]:
#df_res = pd.concat(results)
df_res.to_csv( './Intersections/ACHD_Eagle-All_2024-09_to_2025-08_Plan_Changes_comb.csv')

In [66]:
df_cp=df_raw[(df_raw.Code.isin(list(range(131,150))))&~(df_raw.ID.isin([0,10,254,255]))].drop_duplicates().set_index('TS_start')
df_cp

,Code,ID,Cycle_start,Coord_plan
TS_start,,,,
2024-09-12 06:00:00.700,135,41,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,134,21,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,137,47,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,138,21,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,136,21,2024-09-12 05:57:56.700,254.0
...,...,...,...,...
2025-04-30 19:03:19.700,135,36,2025-04-30 18:59:55.700,5.0
2025-04-30 19:03:19.700,136,21,2025-04-30 18:59:55.700,5.0
2025-04-30 19:03:19.700,137,52,2025-04-30 18:59:55.700,5.0


In [7]:
def plan_changes(df_raw):
    #df_cp=df_raw[(df_raw.Code.isin(list(range(131,150))))&(df_raw.ID!=0)].drop(columns='Coord_plan').drop_duplicates()
    df_cp=df_raw[(df_raw.Code.isin(list(range(131,150))))&~(df_raw.ID.isin([0,10,254,255]))].drop_duplicates().set_index('TS_start')
    df_cp=df_cp.pivot_table(index=df_cp.index, columns='Code', values='ID', aggfunc='first')
    
    df_nocp=df_cp.loc[df_cp[131].isna()]

    
    df_cp=df_cp.sort_values([131,'TS_start'])\
                .rename(columns={131:'Coord_plan',132:'Cycle_len',133:'Offset',
                                134:'Ph1',135:'Ph2',136:'Ph3',137:'Ph4',
                                138:'Ph5',139:'Ph6',140:'Ph7',141:'Ph8'})

    # Add StartTime column: round TS_start index to nearest half hour and format as hhmm ## THIS WILL REQUIRE SIGNIFICANT ADDITIONAL WORK
    #start_times = df_cp.index.round('30min').strftime('%H%M')
    #df_cp['StartTime'] = start_times

    df_cp = merge_complementary_rows_chain(df_cp)

    df_cp = df_cp[(~df_cp.Coord_plan.isna())]

    res=[]
    for i,j in df_cp.Coord_plan.value_counts().sort_index().items():
        for k in range(1, j+1):
            res.append(i + k/100)
    
    df_cp.Coord_plan = res

    return df_cp

In [9]:
def merge_complementary_rows_chain(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    merged_rows = []
    current = df.iloc[0].copy()
    current.name = df.index[0]   # ensure first index is kept

    for i in range(1, len(df)):
        row = df.iloc[i].copy()
        row.name = df.index[i]   # keep row index

        # conflict if both have values and they differ
        conflict_mask = (current.notna() & row.notna() & (current != row))

        if conflict_mask.any():
            # finalize current chain (with its original index)
            merged_rows.append(current)
            # start new chain (with this row’s index)
            current = row
        else:
            # merge into current (union of values), keeping current index
            merged = current.combine_first(row)
            merged.name = current.name
            current = merged

    # finalize last chain
    merged_rows.append(current)

    # return DataFrame with preserved indices
    return pd.DataFrame(merged_rows)

In [10]:
df_coord=df_raw.loc[df_raw.Code==131,['TS_start','ID']]
df_coord=pd.merge(df_coord,df_cp.Coord_plan,how='left',left_on='TS_start',right_index=True)
df_coord=df_coord.groupby('ID').ffill()

In [44]:
df_cp=df_raw[(df_raw.Code.isin(list(range(131,150))))&~(df_raw.ID.isin([0,10,254,255]))].drop_duplicates().set_index('TS_start')
#df_cp=df_cp.pivot_table(index=df_cp.index, columns='Code', values='ID', aggfunc='first')

#df_nocp=df_cp.loc[df_cp[131].isna()]


#df_cp=df_cp.sort_values([131,'TS_start'])\
#            .rename(columns={131:'Coord_plan',132:'Cycle_len',133:'Offset',
#                            134:'Ph1',135:'Ph2',136:'Ph3',137:'Ph4',
#                            138:'Ph5',139:'Ph6',140:'Ph7',141:'Ph8'})

df_cp

,Code,ID,Cycle_start,Coord_plan
TS_start,,,,
2024-09-12 06:00:00.700,135,41,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,134,21,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,137,47,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,138,21,2024-09-12 05:57:56.700,254.0
2024-09-12 06:00:00.700,136,21,2024-09-12 05:57:56.700,254.0
...,...,...,...,...
2025-04-30 19:03:19.700,135,36,2025-04-30 18:59:55.700,5.0
2025-04-30 19:03:19.700,136,21,2025-04-30 18:59:55.700,5.0
2025-04-30 19:03:19.700,137,52,2025-04-30 18:59:55.700,5.0


In [58]:
dfrs = df_raw.loc[df_raw.Code==131, ['TS_start', 'Coord_plan']]\
             .set_index('TS_start')\
             .resample('1T')

dfrs_day = pd.concat([dfrs.first(),dfrs.last()])\
             .reset_index()\
             .sort_values('TS_start')\
             .set_index('TS_start')\
             .dropna()

#dfrs_day = dfrs_day.loc[(dfrs_day.Coord_plan!=dfrs_day.Coord_plan.shift()),:]\
#                   .resample('1440T')\
#                   .apply(lambda x: str(list(x.Coord_plan))[0:-1].replace(' ',''))

df_res = pd.concat([dfrs_day,pd.Series(dfrs_day.index.weekday.values, index=dfrs_day.index)],axis=1)

df_res.columns=['Coord_plans','Weekday']

#df_res.loc[(df_res.Weekday<5),'Weekday']=1

df_gb = df_res.groupby('Weekday').apply(lambda x: x[x.Coord_plans!=x.Coord_plans.shift()]).drop(columns='Weekday')


C:\Users\rhansen\AppData\Local\Temp\ipykernel_56508\937717342.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample('1T')
C:\Users\rhansen\AppData\Local\Temp\ipykernel_56508\937717342.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_gb = df_res.groupby('Weekday').apply(lambda x: x[x.Coord_plans!=x.Coord_plans.shift()]).drop(columns='Weekday')


In [52]:
df_gb = df_gb.reset_index().sort_values('TS_start')

In [64]:
dfrs.first().dropna()

,Coord_plan
TS_start,
2024-09-12 06:00:00,254.0
2024-09-12 06:20:00,1.0
2024-09-12 09:31:00,2.0
2024-09-12 11:32:00,3.0
2024-09-12 12:00:00,3.0
...,...
2025-04-30 12:00:00,3.0
2025-04-30 14:00:00,13.0
2025-04-30 18:00:00,13.0


In [23]:
df=df_raws['Franklin & KCID']
df[(df.Code.isin([1,6,150,151,152])&(df.Coord_plan==1))]

,TS_start,Code,ID,Cycle_start,Coord_plan
9655608,2025-10-27 06:29:16.000,1,2,2025-10-27 06:29:16.000,1.0
9655609,2025-10-27 06:29:16.000,1,6,2025-10-27 06:29:16.000,1.0
9655725,2025-10-27 06:29:38.300,1,4,2025-10-27 06:29:16.000,1.0
9655726,2025-10-27 06:29:38.300,1,8,2025-10-27 06:29:16.000,1.0
9655872,2025-10-27 06:30:00.000,150,2,2025-10-27 06:29:16.000,1.0
...,...,...,...,...,...
14410021,2025-11-11 18:58:50.000,6,6,2025-11-11 18:58:12.600,1.0
14410024,2025-11-11 18:58:54.200,151,2,2025-11-11 18:58:12.600,1.0
14410049,2025-11-11 18:59:15.000,6,2,2025-11-11 18:58:12.600,1.0
14410069,2025-11-11 18:59:21.100,1,8,2025-11-11 18:58:12.600,1.0
